In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import json
import plotly.express as px
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
with open('config/EDA.json') as fh:
    configs = json.load(fh)
    EDA_VIZ_PATH, TB_US_PATH = configs['EDA_VIZ_PATH'], configs['TB_US_PATH']

In [3]:
tb_use  = pd.read_csv(TB_US_PATH)
tb_use.head(2)

,Year,Locationabbr,Locationdesc,Topic,Question,Response,Break_Out,Break_Out_Category,Sample_Size,Data_value,...,Confidence_limit_High,Data_value_unit,Data_value_type,Data_Value_Footnote,LocationID,BreakoutID,BreakOutCategoryID,QuestionID,ResponseID,GeoLocation
0,2019,KS,Kansas,Smokeless Tobacco,"Do you currently use chewing tobacco, snuff, o...",Every day,25-34,Age Group,52,4.31,...,5.60,%,Crude Prevalence,NaN,20,AGE02,CAT3,USENOW3,RESP079,"(38.34774030000045, -98.20078122699965)"
1,2019,KS,Kansas,Smokeless Tobacco,"Do you currently use chewing tobacco, snuff, o...",Some days,Female,Gender,17,0.40,...,0.62,%,Crude Prevalence,NaN,20,SEX2,CAT2,USENOW3,RESP080,"(38.34774030000045, -98.20078122699965)"


In [4]:
# Tranforming sample size to int (no invalid string, checked)
tb_use['Sample_size'] = tb_use['Sample_Size'].apply(lambda x: int(x.replace(',', '')))

In [5]:
# this can be better optimized by first finding total sample size per group and left joining it onto the original table
def impute_subgroup(sub_group):
#     print(f'new subgroup: ')
#     display(sub_group)
    total = sub_group['Sample_size'].sum()
    def impute_dv(row):
        if pd.isna(row['Data_value']):
            if total == 0:
                return 0
            return (row['Sample_size'] / total) *100
        else:
            return row['Data_value']
    sub_group['Data_value'] = sub_group.apply(impute_dv, axis=1)
    return sub_group

In [6]:
# impute data value by manually calculating proportion of sample size (DOF ignore so maybe a bit off in value)
tb_use_cleaned = tb_use.groupby(['Year', 'Locationabbr', 'Break_Out', 'Question']).apply(impute_subgroup)

In [7]:
tb_use_cleaned.to_csv('data/tobacco_use_us_cleaned.csv')

Extensive EDAs are inside the tableau workbook.

# Preliminary

Year range

In [8]:
tb_use['Year'].unique()

array([2019, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2020],
      dtype=int64)

Locationdesc: state abbr + UW (all states and DC (median)) and US (united states). Does not have equal participation

In [9]:
# tb_use['Locationabbr'].value_counts().head()

Topic - Quesion is 1 on 1  
 
- Q: Smoker Status - Four Level Smoking Status (variable calculated from one or more BRFSS questions)  
- A: \['Never smoked', 'Smoke some days', 'Smoke everyday', 'Former smoker'\]  
  
- Q: Smokeless Tobacco - Do you currently use chewing tobacco, snuff, or snus? 
- A: \[Every day, Some days, Not at all\]  

- Q: Current Smoker Status - Adults who are current smokers (variable calculated from one or more BRFSS questions)  
- A: \[Yes, No\]

In [10]:
# tb_use.groupby('Topic')['Response'].unique()['Smoker Status']

In [11]:
# tb_use.groupby('Topic')['Question'].apply(lambda x: len(set(x)))
# tb_use['Topic'].value_counts()
# tb_use['Question'].value_counts()
# state_occur = tb_use['Locationabbr']\
#     .value_counts().to_frame().reset_index()\
#     .rename(columns = {'index':'code', 'Locationabbr':'num_records'})

breakouts

In [12]:
# tb_use.groupby(['Topic', 'Break_Out'])[['Locationabbr']].count().loc['Current Smoker Status']

# Quantifying tobacco culture

territory to exclude: puertorico (too small sample size), virgin island (one data point lmao)

visual clustering:  
- y = overall health awareness
- x = income difference

In [14]:
years = range(2011, 2021)
states = set(tb_use_cleaned['Locationabbr'].unique()) - set({'GU', 'VI', 'PR', 'US', 'UW'})

In [21]:
state = 'AZ'
year = 2011
low_income = 'INCOME1'
high_income = 'INCOME5'
overall = 'BO1'

In [22]:
relavent_year_state = tb_use_cleaned.loc[\
                       (tb_use_cleaned['Locationabbr'] == state) &\
                       (tb_use_cleaned['Year'] == year)&(tb_use_cleaned['QuestionID']=='_RFSMOK3')&\
                       (tb_use_cleaned['Response']=='Yes')]

In [23]:
relavent_year_state.loc[relavent_year_state['Break_Out']=='Overall']

,Year,Locationabbr,Locationdesc,Topic,Question,Response,Break_Out,Break_Out_Category,Sample_Size,Data_value,...,Data_value_unit,Data_value_type,Data_Value_Footnote,LocationID,BreakoutID,BreakOutCategoryID,QuestionID,ResponseID,GeoLocation,Sample_size
17442,2011,AZ,Arizona,Current Smoker Status,Adults who are current smokers (variable calcu...,Yes,Overall,Overall,"1,023",19.3,...,%,Crude Prevalence,NaN,4,BO1,CAT1,_RFSMOK3,RESP046,"(34.86597028, -111.763811277)",1023


In [24]:
relavent_year_state['Data_value'].mean()

20.345904954499495

In [25]:
def state_by_subgroup(year, state, group1=low_income, group2=high_income, overall=overall):
    relavent_year_state = tb_use_cleaned.loc[\
                       (tb_use_cleaned['Locationabbr'] == state) &\
                       (tb_use_cleaned['Year'] == year)&(tb_use_cleaned['QuestionID']=='_RFSMOK3')&\
                       (tb_use_cleaned['Response']=='Yes')]
    name = relavent_year_state['Locationdesc'].iloc[0]
#     print(relavent_year_state['Locationabbr'].unique())
    subgroup_tobacco_use = relavent_year_state.loc[
                       (tb_use_cleaned['BreakoutID'].isin([group1, group2]))\
                    ].set_index('BreakoutID')['Data_value']
    # average proportion of people not currently smokers among all subgroups
    overall_healthiness = 100 - relavent_year_state.loc[relavent_year_state['BreakoutID'] == overall]\
        .set_index('BreakoutID')['Data_value'].loc[overall]
    
    # diff in proportion of people that are smokers
    diff_subgroups_smoking = subgroup_tobacco_use.loc[group1] - subgroup_tobacco_use.loc[group2]
    
    return name, overall_healthiness, diff_subgroups_smoking

In [26]:
def generate_scatter_data(group1, group2, overall):
    cols = {'Year':[], 'Code':[], 'state':[], 'overall_health':[], 'diff_smoke_income':[]}
    for year in years:
        for state in states:
    #         print(state, year)
            try:
                name, x, y = state_by_subgroup(year, state, group1=group1, group2=group2, overall=overall)
            except:
                name, x, y = None, None, None
            cols['Year'].append(year)
            cols['Code'].append(state)
            cols['overall_health'].append(x)
            cols['state'].append(name)
            cols['diff_smoke_income'].append(y)
    df = pd.DataFrame(cols)
    df.to_csv('data/healthy_vs_income_diff.csv')
    return df

In [28]:
scatter = generate_scatter_data(low_income, high_income, overall='AGE01')

In [29]:
scatter.head()

,Year,Code,state,overall_health,diff_smoke_income
0,2011,NV,Nevada,82.6,20.8
1,2011,LA,Louisiana,77.2,15.0
2,2011,MA,Massachusetts,77.4,20.3
3,2011,WI,Wisconsin,77.0,20.1
4,2011,GA,Georgia,75.0,24.5


In [ ]:
progression = scatter\
    .groupby(['Code', 'state'])\
    .apply(lambda x: x['overall_health'].iloc[-1] - x['overall_health'].iloc[0])\
    .rename('progression').to_frame().reset_index()

In [ ]:
progression.head()

In [ ]:
progression.to_csv('data/youth_progression.csv', index=False)

In [ ]:
scatter_overall = generate_scatter_data(group1, group2, overall)
progression_overall = scatter_overall\
    .groupby(['Code', 'state'])\
    .apply(lambda x: x['overall_health'].iloc[-1] - x['overall_health'].iloc[0])\
    .rename('progression').to_frame().reset_index()

In [ ]:
progression_overall.head()

In [ ]:
progression_overall.to_csv('data/overall_progression.csv', index=False)